# A Guide to CoreML on iOS

![alt text](https://www.cygnismedia.com/images/post-images/wwdc17-for-ios-app-developers/apple-core-ml.jpg "Logo Title Text 1")

## Our Demo

We're going to build a simple iMessage app that can detect whether or not a message is spam. With the Messages framework, you can build a standalone iMessage app, or an extension to an iOS app that lets users interact with your iOS app without leaving Messages. We'll build an extension. The Model trained using the SMS Spam Collection Dataset in sklearn and converted using coremltools into .mlmodel format.

![alt text](http://i.imgur.com/a3XkZJR.png "Logo Title Text 1")

## The Stack

### First, some History... 

#### The previous ML frameworks for iOS
Last year, Apple Announced the Metal Performance Shader Convolutional Neural Network (MPSCNN). This implement CNNs using convolutionss, pooling, normalization, and other matrix multiplication operations. They also announced Basic Neural Network Subroutines (BNNS) a collection of math functions that take full advantage of the CPU’s fast vector instructions (also used for CNNs). 

#### Links to documentation
https://developer.apple.com/documentation/accelerate/bnns (BNNs)
https://developer.apple.com/documentation/metalperformanceshaders (MPSCNN)

#### Features
- Both let developers perform inference on convolutional neural networks on iOS devices.
- BNNS runs on the CPU
- MPSCNN runs on the GPU. 
- Sometimes using the CPU is faster, sometimes using the GPU is faster.
- For training GPU always wins,but for inference sometimes CPU can be faster

#### Where we are today
This year we get lots of additions to Metal (the framework that allows direct access to on-device GPUs), a new computer vision framework, and Core ML: a toolkit that makes it really easy to put ML models into your app.

![alt text](https://docs-assets.developer.apple.com/published/bc34b3e6c2/db81e861-1e06-4d14-8915-90707d9b114c.png "Logo Title Text 1")

- CoreML is built on its previous 2 released ML libraries
- CoreML is the foundation for domain-specific frameworks and functionality.
- The 3 domains are Computer vision (image analysis), Foundation aka Natural Language Processing, and GamePlayKit (evaluating learned decison trees)
- So you can take a CoreML model and pass it to any of these 3 APIs (yay modularity)
- ML primitives > ML framework > domain specific ML frameworks > your app

## The Development Workflow

![alt text](https://docs-assets.developer.apple.com/published/72e22672fd/c35ebf2d-ee94-4448-8fae-16420e7cc4ed.png "Logo Title Text 1")

#### This can be a 2 step process if you use a CoreML model from Apple
1. loading a trained model
2. making predictions

![alt text](https://cdn-images-1.medium.com/max/1600/1*HhVGMZCEtPN-lfwjSQpkRw.png "Logo Title Text 1")

#### Or it can be a 3 step process if you need to convert a third party model into a CoreML model
1. Convert Model (via coremltools,a a Python package designed to help generate an .mlmodel file that Xcode can use)
2. Load the Model
3. Make Predictions

#### The .mlmodel file is like the PDF of Machine Learning Models. Apple has created a standard format for its devices.

- You can find popular models that apple has already converted into it's format here https://developer.apple.com/machine-learning/ 
- Else, here is an amazing list https://github.com/likedan/Awesome-CoreML-Models
- The file format describes the layers in your model, the input and outputs, the class labels, any preprocessing that needs to happen on the data, and the learned parameters (the weights and biases).

![alt text](https://docs-assets.developer.apple.com/published/bc34b3e6c2/6b4f8b26-1cd0-4d9e-8b54-54dac16a808c.png "Logo Title Text 1")

#### Sample 

```swift
let model = ResNet50()
let pixelBuffer: CVPixelBuffer = /* your image */

if let prediction = try? model.prediction(image: pixelBuffer) {
  print(prediction.classLabel)
}
```
That's it!

## Pros vs Cons of CoreML


![alt text](https://cdn-images-1.medium.com/max/1600/1*7wyTjs8euvFaqiAC8NWMzA.png "Logo Title Text 1")


### Pros

- Optimized for on-device performance, which minimizes memory footprint and power consumption. 
- On-device means privacy of user data 
- On-Device means functional predictions even without an internet connection
- It decides for itself whether to run the model on the CPU or the GPU (or both)
- Because it can use CPU, you can run it from the iOS simulator (which doesn't support GPU)
- It supports many models
  * support vector machines (SVM)
  * tree ensembles such as random forests and boosted trees
  * linear regression and logistic regression
  * neural networks: feed-forward, convolutional, recurrent

![alt text](http://upload-images.jianshu.io/upload_images/519579-01796e57a93fd837.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240 "Logo Title Text 1")


#### Cons

- Native support for Supervised models only (no unsupervised or reinforcement learning, but could use 'generic' NN type potentially)
- No training on device, only inference 
- If CoreML does not support a certain layer type, you can’t use it. Currently impossible to extend Core ML with your own layer types
-  Core ML conversion tools only support specific versions of a limited number of training tools (no tensorflow)
- You cannot look at the output produced by intermediate layers, you only get the prediction
- Only supports regression & classification. No clustering, ranking, dimensionality reduction, etc.)
- no federated learning

### My biggest wish for CoreML - add federated learning!

![alt text](https://1.bp.blogspot.com/-K65Ed68KGXk/WOa9jaRWC6I/AAAAAAAABsM/gglycD_anuQSp-i67fxER1FOlVTulvV2gCLcB/s640/FederatedLearning_FinalFiles_Flow%2BChart1.png "Logo Title Text 1")


A Your phone personalizes the model locally, based on your usage 
B Many users' updates are aggregated 
C to form a consensus change to the shared model, after which the procedure is repeated.


### TL;DR CoreML is super simple to use but limited in its functionality. If you want full control, you’re going to have to DIY with Metal Performance Shaders or the Accelerate framework — or both!


### Don't worry, there are other ways to do ML on iOS

http://alexsosn.github.io/ml/2015/11/05/iOS-ML.html#cv


# Our steps 

First, in Python....

1. Import our dataset
2. Train several models using Scikit-learn
3. Convert the best model into an .mlmodel file

Then, in Swift

1. Drag and drop our dataset and trained .mlmodel into our project
2. Write the basic prediction code
3. Write the TF-IDF code (short for term frequency–inverse document frequency, a a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus)
4. Demo!


## Dependencies

In [1]:
#Sklearn and numpy
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

### CoreMLtools doesnt run on python 3 so well need to install it via python 2
```
brew install pyenv
pyenv install 2.7.12
pyenv global 2.7.12
pyenv rehash
pip install coremltools
```

In [ ]:
import coremltools

## Import Data

In [ ]:
#labeled data http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/

# Reading in and parsing data
raw_data = open('SMSSpamCollection.txt', 'r')
sms_data = []
for line in raw_data:
    split_line = line.split("\t")
    sms_data.append(split_line)

# Splitting data into messages and labels and training and test
sms_data = np.array(sms_data)
X = sms_data[:, 1]
y = sms_data[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=22)

print X_train

## Train Models for classification

3 models
- Multinomial Naive Bayes (A specialized version of Naive Bayes that is designed more for text documents. Whereas simple naive Bayes would model a document as the presence and absence of particular words, multinomial naive bayes explicitly models the word counts and adjusts the underlying calculations to deal with in)
![alt text](https://nlp.stanford.edu/IR-book/html/htmledition/img897.png "Logo Title Text 1")
- Support Vector Machine
![alt text](http://docs.opencv.org/2.4/_images/optimal-hyperplane.png "Logo Title Text 1")
- Random Forest 
![alt text](https://i.ytimg.com/vi/ajTc5y3OqSQ/hqdefault.jpg "Logo Title Text 1")


2 vectoriziation strategies
- count vectorizer (Convert a collection of text documents to a matrix of token counts) 
- TF-IDF vectorizers (Convert a collection of raw documents to a matrix of TF-IDF features.) 

![alt text](http://dovgalecs.com/blog/wp-content/uploads/2012/03/img131.gif "Logo Title Text 1")

In [ ]:
# Building Pipelines
pipeline_1 = Pipeline([('vect', CountVectorizer()),('clf', MultinomialNB())])
pipeline_2 = Pipeline([('vect', TfidfVectorizer()),('clf', MultinomialNB())])
pipeline_3 = Pipeline([('vect', CountVectorizer()),('clf', LinearSVC())])
pipeline_4 = Pipeline([('vect', TfidfVectorizer()),('clf', LinearSVC())])
pipeline_5 = Pipeline([('vect', CountVectorizer()),('clf', RandomForestClassifier())])
pipeline_6 = Pipeline([('vect', TfidfVectorizer()),('clf', RandomForestClassifier())])
pipelines = [pipeline_1, pipeline_2, pipeline_3, pipeline_4, pipeline_5, pipeline_6]

# Performing classification and calculating accuracy
for pipeline in pipelines:
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    print(classification_report(y_test, y_pred, target_names=["ham", "spam"]))

## SVC + tfidf wins! Lets Save trained model as .mlmodel

In [9]:
# Creating and saving an .mlmodel file and a list of words
vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(X)
words = open('words_ordered.txt', 'w')
for feature in vectorizer.get_feature_names():
    words.write(feature.encode('utf-8') + '\n')
words.close()
model = LinearSVC()
model.fit(vectorized, y)
coreml_model = coremltools.converters.sklearn.convert(model, "message", 'label')
coreml_model.save('MessageClassifier.mlmodel')